## Image -> Text Task

Goal: Set up a pipeline to Claude to identify non-text parts of the image (shape, icongraphy, etc)

### Prompts:
Feel free to change or add more!

In [ ]:
ICON_PROMPT = "Hi! Can you identify the iconography of this gravestone? Most of the icongraphy should be towards the top of the stone. " \
                "If there is no icongoraphy, just say None. Please only return exactly what the iconography is. Do not say anything else in your answer."
SHAPE_PROMPT = "Test"
TRANSCRIPTION_PROMPT = "Hi! Can you transcribe the text on this gravestone? Please deliminate each line of the transcription with a hyphen. " \
                        "Please only return the transcription. Do not say anything else in your answer."
INFO_PROMPT = "Test"

PROMPTS = [ICON_PROMPT, TRANSCRIPTION_PROMPT]
COLUMNS = ["Image_Name", "Iconography Description", "Claude Transcription"]

INPUT_FOLDER = folder_path = '../data/examples/'

# Note there is a 5MB limit 
# It took 5 minutes to run 38 images

### Imports + Set up Claude API

In [10]:
import requests
import json
import os
import base64
from pathlib import Path
import pandas as pd
import io
from PIL import Image

def get_api_key(file): 
    """
    Get the API Key
    
    Args:
        file (str): Path to the credentials file
    
    Returns:
        str: API Key
    """
    with open(file, 'r') as f:
        return f.read().strip()

API_KEY = get_api_key("credentials.txt")
API_URL = "https://api.anthropic.com/v1/messages"
headers = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY,
    "anthropic-version": "2023-06-01"
}

### Functions to prompt Claude

In [11]:
# Debug function
def debug_request(data):
    """Print request details for debugging"""
    print("=== DEBUG INFO ===")
    print(f"URL: {API_URL}")
    print(f"Method: POST")
    print("Headers:")
    for key, value in headers.items():
        if key == "x-api-key":
            print(f"  {key}: {value[:10]}...")  # Only show first 10 chars
        else:
            print(f"  {key}: {value}")
    print(f"Data keys: {list(data.keys())}")
    print(f"Model: {data.get('model')}")
    print(f"Message type: {type(data.get('messages', [{}])[0].get('content'))}")
    print("===================")


def encode_image(image_path):
    """
    Encode an image to base64 string
    
    Args:
        image_path (str): Path to the image file
    
    Returns:
        tuple: (base64_string, media_type) or (None, None) if error
    """
    try:
        # Get file extension to determine media type
        file_ext = Path(image_path).suffix.lower()
        media_type_map = {
            '.jpg': 'image/jpeg',
            '.jpeg': 'image/jpeg',
            '.png': 'image/png',
            '.gif': 'image/gif',
            '.webp': 'image/webp'
        }
        
        if file_ext not in media_type_map:
            print(f"Unsupported image format: {file_ext}")
            return None, None
        
        with open(image_path, 'rb') as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
            return encoded_string, media_type_map[file_ext]
    
    except Exception as e:
        print(f"Error encoding image: {e}")
        return None, None


def call_claude(prompt, image_path = None, model="claude-sonnet-4-20250514", debug="False"):
    """
    Call Claude API with a message and optional image
    
    Args:
        message (str): The message to send to Claude
        image_path (str): Path to image file (optional)
        model (str): The model to use (default: claude-sonnet-4-20250514)
    
    Returns:
        dict: API response
    """
    
    # Set content to the prompt
    content = [{"type": "text", "text": prompt}]

    # Add and encode an image if present 
    if image_path:
        encoded_image, media_type = encode_image(image_path)
        if encoded_image:
            content.insert(0, {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": media_type,
                    "data": encoded_image
                }
            })
        else:
            print("Failed to encode image, continuing with text only")
    
    # Query to Claude
    data = {
        "model": model,
        "max_tokens": 1024,
        "messages": [
            {
                "role": "user",
                "content": content
            }
        ]
    }

    # Error Handleing and Debug Mode
    if debug:
        debug_request(data)
    try:
        response = requests.post(API_URL, headers=headers, json=data, timeout=30)
        
        if debug:
            print(f"Response status: {response.status_code}")
            print(f"Response headers: {dict(response.headers)}")
        
        # Print response content for debugging
        if response.status_code != 200:
            print(f"Error response: {response.text}")
        
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f"Error calling API: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response status: {e.response.status_code}")
            print(f"Response text: {e.response.text}")
        return None


In [12]:
def list_files_in_folder(folder_path):
    """
    Get the names of all the files in a folder
    
    Args:
        folder_path (str): Path to folder
    
    Returns:
        list[str]: list of file names
    """
    try:
        files = os.listdir(folder_path)
        
        # Filter out directories, keeping only files
        file_names = [f for f in files if os.path.isfile(os.path.join(folder_path, f))]
        
        return file_names
    
    # Errors
    except FileNotFoundError:
        print(f"The folder at {folder_path} does not exist.")
        return []
    except PermissionError:
        print(f"Permission denied to access the folder at {folder_path}.")
        return []


In [13]:
def gravestone_info(files):
    """
    Call Claude 
    
    Args:
        files list(str): Names of the images
    
    Returns:
        df(DataFrame): Dataframe with the columns specified in constants. Also exports it to output.
    """

    all_results = []

    for image in files:

        image_result = [image]
        for prompt in PROMPTS:
        # Call Claude
            result = call_claude(prompt, folder_path + image, debug=True)
            image_result.append(result['content'][0]['text'])
        # Extract Text
        all_results.append(image_result)

    # Put in a dataframe
    df = pd.DataFrame(all_results, columns=COLUMNS)
    df.to_csv('../data/output.csv', index=False)

    return df
        

### Run the code here

In [14]:
# Get all the file paths and run the queries
files = list_files_in_folder(INPUT_FOLDER)
gravestone_info(files)

=== DEBUG INFO ===
URL: https://api.anthropic.com/v1/messages
Method: POST
Headers:
  Content-Type: application/json
  x-api-key: sk-ant-api...
  anthropic-version: 2023-06-01
Data keys: ['model', 'max_tokens', 'messages']
Model: claude-sonnet-4-20250514
Message type: <class 'list'>
Response status: 200
Response headers: {'Date': 'Thu, 17 Jul 2025 16:59:11 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'anthropic-ratelimit-input-tokens-limit': '30000', 'anthropic-ratelimit-input-tokens-remaining': '28000', 'anthropic-ratelimit-input-tokens-reset': '2025-07-17T16:59:12Z', 'anthropic-ratelimit-output-tokens-limit': '8000', 'anthropic-ratelimit-output-tokens-remaining': '8000', 'anthropic-ratelimit-output-tokens-reset': '2025-07-17T16:59:11Z', 'anthropic-ratelimit-requests-limit': '50', 'anthropic-ratelimit-requests-remaining': '49', 'anthropic-ratelimit-requests-reset': '2025-07-17T16:59:10Z', 'anthropic-ratelimit-tokens-limit': '380

,Image_Name,Iconography Description,Claude Transcription
0,_DSC0437.jpeg,None,ERECTED\n- to the Memory of\n- [partially visi...
1,_DSC0421.jpeg,None,"In Memory\nof\nSARAH THURBER\nBENSON,\nrelict ..."
2,.DS_Store,I don't see an image attached to your message....,I don't see any image attached to your message...
3,_DSC0420.jpeg,None,In Memory\nof\nFRANCES BENSON\neldest daughter...
4,_DSC0416.jpeg,None,"In Memory\n-of\n-HENRY E. BENSON,\n-youngest s..."
5,_DSC0441.jpeg,None,"ASAHEL F. PROCTOR\n-BORN\n-AUG. 24, 1848.\n-DI..."
6,_DSC0457.jpeg,Crown,SACRED\n- to the memory of\n- MRS. SARAH OLNEY...
7,_DSC0466.jpeg,Three circular rosettes,"PAULINA,-\ndaughter of-\nMr Warren & Mrs Freel..."
8,_DSC0470.jpeg,None,ELLEN H. CUNLIFF\n-\nDAUGHTER OF\n-\nJOSEPH & ...
9,_DSC0446.jpeg,None,In Memory of-\nMr Charles Easterbrooks-\nwho d...
